#### Load and Prepare Data

In [1]:
import pandas as pd
import numpy as np

# Load data from Phase 1
df = pd.read_csv('processed_crypto_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create a proxy for Market Cap (Price * Volume) to represent "Trading Interest"
df['mcap_proxy'] = df['close'] * df['volume']

# Pivot data for correlation analysis (Rows: Timestamp, Columns: Symbols, Values: Price)
price_pivot = df.pivot(index='timestamp', columns='symbol', values='close')

print("Data loaded and pivoted. Ready for KPI calculation.")

Data loaded and pivoted. Ready for KPI calculation.


#### Calculate Sector Dominance (The "Market Share" KPI)

In [2]:
# Group by timestamp and sector to get total daily volume-cap per sector
sector_daily = df.groupby(['timestamp', 'sector'])['mcap_proxy'].sum().unstack()

# Calculate Total Market Interest per day (sum of all sectors)
total_daily = sector_daily.sum(axis=1)

# Calculate Dominance % (Sector Cap / Total Cap)
sector_dominance = sector_daily.div(total_daily, axis=0) * 100

print("Sector Dominance calculated (Percentage of total liquid interest).")
sector_dominance.tail()

Sector Dominance calculated (Percentage of total liquid interest).


sector,DeFi,Layer_1,Meme
timestamp,,,
2026-01-23 00:00:00+00:00,3.042956,92.755161,4.201883
2026-01-24 00:00:00+00:00,1.181694,94.412053,4.406252
2026-01-25 00:00:00+00:00,0.862275,94.989411,4.148314
2026-01-26 00:00:00+00:00,3.181017,95.205533,1.613450
2026-01-27 00:00:00+00:00,0.104021,99.284219,0.611760


#### Rolling BTC Correlation (The "Competitive Intelligence" KPI)

In [5]:
# Calculate daily percentage returns
returns = price_pivot.pct_change()

# Calculate 30-day rolling correlation of all assets against BTC/USD
# BTC is the benchmark for the entire industry
btc_corr = returns.rolling(window=30).corr(returns['BTC/USD'])

# Remove the BTC/USD column from the result (it will always be 1.0)
btc_corr = btc_corr.drop(columns=['BTC/USD'])

print("30-Day Rolling BTC Correlation calculated.")
btc_corr.tail()

30-Day Rolling BTC Correlation calculated.


C:\Users\Eldu\AppData\Local\Temp\ipykernel_16140\1078956642.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = price_pivot.pct_change()


symbol,AAVE/USD,DOGE/USD,ETH/USD,MKR/USD,PEPE/USD,SHIB/USD,SOL/USD,UNI/USD
timestamp,,,,,,,,
2026-01-23 00:00:00+00:00,0.771769,0.715386,0.904094,NaN,0.543015,0.647258,0.850811,0.758928
2026-01-24 00:00:00+00:00,0.774058,0.716451,0.903216,NaN,0.541592,0.646229,0.852748,0.765179
2026-01-25 00:00:00+00:00,0.801119,0.727885,0.913483,NaN,0.557948,0.661414,0.868614,0.776984
2026-01-26 00:00:00+00:00,0.809069,0.729775,0.916300,NaN,0.565384,0.664165,0.874053,0.782831
2026-01-27 00:00:00+00:00,0.813861,0.729149,0.914879,NaN,0.566960,0.665462,0.873026,0.814797


#### Generating "Actionable Recommendations"

In [7]:
# Identify the fastest growing sector in dominance over the last 30 days
growth = sector_dominance.iloc[-1] - sector_dominance.iloc[-30]
top_sector = growth.idxmax()

# Identify the least correlated asset (The "De-coupler")
avg_corr = btc_corr.iloc[-1].sort_values()
top_decoupler = avg_corr.idxmin()

# Save for Streamlit App
sector_dominance.to_csv('sector_dominance.csv')
btc_corr.to_csv('btc_correlation.csv')

print(f"BI INSIGHTS")
print(f"Trending Sector: {top_sector} (Growth: {growth.max():.2f}%)")
print(f"Strategic Opportunity: {top_decoupler} is least correlated to BTC (Value: {avg_corr.min():.2f})")

BI INSIGHTS
Trending Sector: Layer_1 (Growth: 8.61%)
Strategic Opportunity: PEPE/USD is least correlated to BTC (Value: 0.57)
